# Trabalho prático 2
### Processamento de Sinais Multimédia
####  Alunos: Diana Oliveira 52651; Beatriz Jesus 52749; Guilherme Cavalcanti 53077

In [ ]:
import numpy as np
from scipy import signal as ss
import scipy.io.wavfile as wav 
from IPython.display import Audio, display 

##### Exercício 1

In [ ]:
# --------- Parâmetros Globais ---------
Fs = 48000  # Frequência de amostragem
temperatura = 20  # Temperatura em graus Celsius
recetor = (10,4) # Coordenadas do recetor (x,y) em metros
fonte = (8,7) # Coordenadas da fonte (x,y) em metros
S1 = (8,13) # reflexão da parede horizontal absorção=0.5
S2 = (-7,7) # reflexão da parede vertical absorção=0.3
a = [0.5, 0.3]  # Coeficientes de absorção das paredes [parede vertical, parede horizontal]
c = 331.5 * np.sqrt(1 + temperatura/273) # Velocidade do som no ar em m/s 

# --------- MUTES ---------
#campo_direto_mute = False
mute1 = False

# --------- Converter Aúdio ---------
# O aúdio utilizado já tem 16 bits mas fs=44100 Hz --> Tem que converter para Fs=48000 Hz
fs_audio_org, audio_org = wav.read("Borderline_UnmasteredWAV.wav")

num_samples_new = int(len(audio_org) * Fs / fs_audio_org) # Número de amostras do áudio novo
audio_resampled = ss.resample(audio_org, num_samples_new, axis=0) # Resample do áudio para a nova frequência de amostragem 
audio_resampled = audio_resampled.astype(np.int16) # Converter para 16 bits, senão fica a 64 bits

wav.write("AudioConvertido.wav", Fs, audio_resampled)

# --------- Ler Aúdio Convertido  ---------
_ , audio = wav.read("AudioConvertido.wav")

In [ ]:
# --------(d) Campo total = Campo direto + Primeiras reflexões + Campo Reverberante ---------
distancia_recetor_fonte = np.sqrt((np.abs(recetor[0])-np.abs(fonte[0]))**2 + (np.abs(recetor[1])-np.abs(fonte[1]))**2)  # Distância entre fonte e recetor
distancia_recetor_S1 = np.sqrt((np.abs(recetor[0])-np.abs(S1[0]))**2 + (np.abs(recetor[1])-np.abs(S1[1]))**2)  # Distância entre recetor e S1
distancia_recetor_S2 = np.sqrt((np.abs(recetor[0])-np.abs(S2[0]))**2 + (np.abs(recetor[1])-np.abs(S2[1]))**2)  # Distância entre recetor e S2

# --------- Campo Direto ---------
t1 = distancia_recetor_fonte / c  # Tempo de atraso em segundos
m1 = int(t1 * Fs)  # Número de amostras de atraso
a1 = audio[:,0] * (1/distancia_recetor_fonte) # ganho (atenuação)

# --------- 1º Reflexão ---------
t2 = distancia_recetor_S1 / c  # Tempo de atraso em segundos
m2 = int(t2 * Fs)  # Número de amostras de atraso
a2 =  audio[:,0] * (1-a[0]) * (1/distancia_recetor_S1)  # ganho (coeficiente absorção * atenuação)

# --------- 2º Reflexão ---------
t3 = distancia_recetor_S2 / c  # Tempo de atraso em segundos
m3 = int(t3 * Fs)  # Número de amostras de atraso
a3 = audio[:,0] * (1-a[1]) * (1/distancia_recetor_S2) # ganho (coeficiente absorção * atenuação pela distância)


# Aplicar o atraso a cada um dos audios

atraso1 = np.zeros((m1))
a1 = np.concatenate((atraso1, a1), axis=0)# ganho do campo direto

atraso2 = np.zeros((m2))    
a2 = np.concatenate((atraso2, a2), axis=0) # ganho da 1ª reflexão

atraso3 = np.zeros((m3))
a3 = np.concatenate((atraso3, a3), axis=0)# ganho da 2ª reflexão

# Determinar o tamanho máximo
max_len = max(len(a1), len(a2), len(a3))

# Preencher zeros no final de cada array para igualar tamanhos
a1 = np.pad(a1, (0, max_len - len(a1)), mode='constant')
a2 = np.pad(a2, (0, max_len - len(a2)), mode='constant')
a3 = np.pad(a3, (0, max_len - len(a3)), mode='constant')


campo = a1 + a2 + a3 #campo direto, 1ª e 2ª reflexão



# --------- RT60 ---------

mi = int(0.03 * Fs)   # atraso de 30 milissegundos 

def reverb(campo, RT60):
    ganho = 10**((-3 * mi) / (RT60)) # 20*log10(g) / (m*T) = -60 / Tr  => log10(g) = -3*m*T / Tr , onde tr é rt60 e o T é o peirodo de amostragem 1/Fs
    bks = [1]
    aks = np.zeros(mi + 1)
    aks[0] = 1
    aks[-1] = -ganho
    return ss.lfilter(bks, aks, campo)



# Aplicar a reverberação para diferentes valores de RT60
#campo0  = reverb(campo, 0)
campo05 = reverb(campo, 0.5)
campo2  = reverb(campo, 2)
campo10 = reverb(campo, 10)
    

# Normalizar os sinais
#campo_total_0 = campo0/np.max(np.abs(campo0))
campo_total_05 = campo05/np.max(np.abs(campo05))
campo_total_2 = campo2/np.max(np.abs(campo2))
campo_total_10 = campo10/np.max(np.abs(campo10))


# --------- Gravar e Reproduzir 10 segundos do Aúdio ---------
filename_exercício1 = ["Exercício1_RT60_0.wav", "Exercício1_RT60_0.5.wav", "Exercício1_RT60_2.wav", "Exercício1_RT60_10.wav"]
#wav.write(filename_exercício1[0], Fs, (campo_total_0 * 32767).astype(np.int16))
wav.write(filename_exercício1[1], Fs,  (campo_total_05).astype(np.int16))
wav.write(filename_exercício1[2], Fs,  (campo_total_2).astype(np.int16))
wav.write(filename_exercício1[3], Fs,  (campo_total_10).astype(np.int16))



print("Aúdio Original")
display(Audio("AudioConvertido.wav", rate=Fs))
#print("RT60 = 0s")
#display(Audio(campo_total_0[:10*Fs], rate=Fs))
print("RT60 = 0.5s")
display(Audio(campo_total_05[:10*Fs], rate=Fs))
print("RT60 = 2s")
display(Audio(campo_total_2[:10*Fs], rate=Fs))
print("RT60 = 10s")
display(Audio(campo_total_10[:10*Fs], rate=Fs))

In [ ]:

# --------- (c) Campo Reverberante ---------
m = int(0.03 * Fs)   # atraso de 30 milissegundos 

def reverb(campo, RT60):
    ganho = 10**((-3 * m* (1/Fs)) / (RT60)) # 20*log10(g) / (m*T) = -60 / Tr  => log10(g) = -3*m*T / Tr , onde tr é rt60 e o T é o peirodo de amostragem 1/Fs
    bks = [1]
    aks = np.zeros(m + 1)
    aks[0] = 1
    aks[-1] = -ganho
    return ss.lfilter(bks, aks, campo)


# Aplicar a reverberação para diferentes valores de RT60
#Como a reverberaçao esta a ser aplicada num audio, a reverberaçao nao acaba
#campo05 = reverb(audio[:,0], 0.5)
#campo2  = reverb(audio[:,0], 2)
#campo10 = reverb(audio[:,0], 10)

# Para se observar o RT60, aplica-se a um impulso
impulso = np.zeros(20 * Fs)
impulso[0] = 1

teste1 = reverb(impulso, 0.5)
teste2 = reverb(impulso, 2)
teste3 = reverb(impulso, 10)
    
t = np.arange(len(impulso)) / Fs 
plt.plot(t, teste1, label='RT60 = 0.5 s', color='blue')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()


plt.plot(t, teste2, label='RT60 = 2 s', color='orange')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()


plt.plot(t, teste3, label='RT60 = 10 s', color='green')
plt.xlim(0, 11)  # primeiros 11s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()






In [ ]:
# --------- (a) Campo Direto ---------
t1 = distancia_recetor_fonte / c  # Tempo de atraso em segundos
m1 = int(t1 * Fs)  # Número de amostras de atraso
a1 = (1/distancia_recetor_fonte) # ganho (atenuação)
# ABKS E BKS
aks = 1
bks = np.zeros(m1 + 1)
bks[m1] = a1
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro


display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# --------- (b) 1ª Reflexão ---------
t2 = distancia_recetor_S1 / c  # Tempo de atraso em segundos
m2 = int(t2 * Fs)  # Número de amostras de atraso
a2 = (1-a[0]) * (1/distancia_recetor_S1)  # ganho (coeficiente absorção * atenuação)

# ABKS E BKS
aks = [1]
bks = np.zeros(m2 + 1)
bks[m2] = a2
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro

display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()

In [ ]:
# --------- 2ª Reflexão ---------
t3 = distancia_recetor_S2 / c  # Tempo de atraso em segundos
m3 = int(t3 * Fs)  # Número de amostras de atraso
a3 = (1-a[1]) * (1/distancia_recetor_S2)  # ganho (coeficiente absorção * atenuação)

# ABKS E BKS
aks = [1]
bks = np.zeros(m3 + 1)
bks[m3] = a3
campo = ss.lfilter(bks, aks, audio[:,0]) # Teve que ser mono para não dar erro


display(Audio(campo[:10*Fs], rate=Fs))
#plt.plot(campo)
#plt.xlim(0, 3*Fs)
t = np.arange(len(campo)) / Fs 
plt.figure(figsize=(12,4))
plt.plot(t, campo, label='Campo Direto')
plt.xlim(0, 1.0)  # primeiros 2s, por exemplo
plt.legend()
plt.xlabel('Tempo [s]')
plt.ylabel('Amplitude')
plt.show()
